In [1]:
import glob
import re
from collections import Counter
import json
from subprocess import Popen, PIPE
import os

In [3]:
# Only consider logFiles for whom A DISCREPANCY
# noDiscrepancies = []
# fileToDisc = {}
# for f in glob.glob('/mnt/6TB/dockerMLcerts/attached_dir/all_results/testing-results/v3-experiments-extra/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/json/*'):
#     with open(f, "r") as fileO:        
#         data = json.load(fileO)
#         this_pattern = ""
#         for elem in data:
#             this_pattern += str((int(data[elem] != 0)))

#         # if this_pattern != "00001":
#         #     continue
            
#         if this_pattern == "11111" or this_pattern == "00000":
#             continue

#         fileToDisc[f] = this_pattern
        
#         # OpenSSL passes but someone else rejects
# #         if this_pattern[0] != '0':
# #             continue
        
#         noDiscrepancies.append(f)
            
# print("Total disc: ", len(noDiscrepancies))
# print(noDiscrepancies[0])
# print("\n")

# Parallel version of above
import glob
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

log_dir = '/mnt/6TB/dockerMLcerts/attached_dir/all_results/testing-results/v3-experiments-extra/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/json/*'
log_files = glob.glob(log_dir)

noDiscrepancies = []
fileToDisc = {}

def process_file(f):
    try:
        with open(f, "r") as fileO:
            data = json.load(fileO)
            this_pattern = ""
            for elem in data:
                this_pattern += str((int(data[elem] != 0)))

            if this_pattern == "11111" or this_pattern == "00000":
                return None
                
            return f, this_pattern
    except Exception as e:
        print(f"Error processing {f}: {e}")
        return None

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_file, f): f for f in log_files}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing files"):
        result = future.result()
        if result:
            f, pattern = result
            fileToDisc[f] = pattern
            noDiscrepancies.append(f)

print("Total disc: ", len(noDiscrepancies))
print("\n")

# FRANKENCERTS
# noDiscrepancies = []
# for f in glob.glob('/tmp/XYZ/all-results/June2022/matrixSSL-fixed/testing-results/PRIORWORKS/frankencerts-v1/json/*'):
#     with open(f, "r") as fileO:
#         data = json.load(fileO)
#         this_pattern = ""
#         for elem in data:
#             this_pattern += str((int(data[elem] != 0)))
        
#         #if this_pattern == "00000":
#             #print(f)
#         noDiscrepancies.append(f)
            
# print("Total 00000: ", len(noDiscrepancies))
# print(noDiscrepancies[0])
# print("\n")

Processing files: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 591466/591466 [07:02<00:00, 1400.07it/s]

Total disc:  152923




In [4]:
print(len(noDiscrepancies))
print(noDiscrepancies[0])
print(len(fileToDisc))
print(fileToDisc[noDiscrepancies[0]])

152923
/mnt/6TB/dockerMLcerts/attached_dir/all_results/testing-results/v3-experiments-extra/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/json/output.84737285-708b-4d3f-ab83-55e1eb2f0dbc.pem.json
152923
01010


In [5]:
import subprocess
import json
import collections
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

zlint_data_path = '/mnt/6TB/dockerMLcerts/attached_dir/zlintResults/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/'

all_errors = []
all_errors_source = {}
file_to_error = {}
no_zlint = []

def process_zlint(nD):
    zlint_file_path = zlint_data_path + "output." + nD.split("output.")[1].replace(".json", ".zlint")
    if not os.path.isfile(zlint_file_path):
        return ("no_zlint", nD)
    
    local_errors = []

    try:
        with open(zlint_file_path) as nDO:
            for nDO_l in nDO:
                for xx in nDO_l.split("time="):
                    if xx.startswith('{"raw":'):
                        l = json.loads(xx)
                        if l['zlint']['errors_present']:
                            assert len(l['zlint']['lints']) == 267
                            for lint, lint_data in l['zlint']['lints'].items():
                                if lint_data['result'] == 'error':
                                    local_errors.append(lint)
                                    
    except Exception as e:
        print(f"Error reading {zlint_file_path}: {e}")
    
    return ("zlint", nD, local_errors)

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_zlint, nD): nD for nD in noDiscrepancies}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing zlint files"):
        result = future.result()
        if result[0] == "no_zlint":
            no_zlint.append(result[1])
        elif result[0] == "zlint":
            _, zlint_file_path, local_errors = result
            all_errors.extend(local_errors)
            
            file_to_error[zlint_file_path] = local_errors

print(collections.Counter(all_errors))
print("Number of certs unable to run zlint on:", len(no_zlint))

# import subprocess
# import json
# import collections
# import os

# zlint_data_path = '/mnt/6TB/dockerMLcerts/attached_dir/zlintResults/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/'

# all_errors = []
# all_errors_source = {}
# file_to_error = {}
# no_zlint = []
# for nD in noDiscrepancies:
#     nD_data_path_zlint = zlint_data_path + "output." + nD.split("output.")[1].replace(".json", ".zlint") 
    
#     #result = subprocess.run(['/tmp/XYZ/zcertificate_0.0.1_Linux_x86_64/zcertificate', nD_data_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     #if result.stdout:
#     if os.path.isfile(nD_data_path_zlint):
#         with open(nD_data_path_zlint) as nDO:
#             #print(result.stdout + b"\n", nD_data_path)
#             for nDO_l in nDO.readlines():
#                 for xx in nDO_l.split("time="):
#                     if xx.startswith('{"raw":'):
                        
#                         l = json.loads(xx)
                        
#                         if l['zlint']['errors_present'] == True:
#                             assert len(l['zlint']['lints']) == 267
#                             for lint in l['zlint']['lints']:
#                                 if l['zlint']['lints'][lint]['result'] == 'error':
#                                     #print(lint, l['zlint']['lints'][lint])
#                                     all_errors.append(lint)

#                                     if lint not in all_errors_source:
#                                         all_errors_source[lint] = []

#                                     all_errors_source[lint].append(nD_data_path_zlint)
                                    
#                                     file_to_error[nD_data_path_zlint] = lint

#     else:
#         no_zlint.append(nD)
    
# print(collections.Counter(all_errors))
# print("Number of certs unable to run zlint on:", len(no_zlint))

Processing zlint files: 100%|█████████████████████████████████████████████████████████████████████████████████████| 152923/152923 [02:51<00:00, 893.92it/s]

Counter({'e_tbs_signature_rsa_encryption_parameter_not_null': 101815, 'e_spki_rsa_encryption_parameter_not_null': 101373, 'e_sub_cert_aia_does_not_contain_ocsp_url': 9197, 'e_ext_san_missing': 9039, 'e_sub_cert_aia_missing': 8837, 'e_sub_cert_cert_policy_empty': 8794, 'e_sub_cert_certificate_policies_missing': 8794, 'e_subject_common_name_not_from_san': 8499, 'e_ext_authority_key_identifier_no_key_identifier': 8492, 'e_ext_authority_key_identifier_missing': 8025, 'e_sub_ca_certificate_policies_missing': 6844, 'e_sub_ca_aia_missing': 6695, 'e_sub_ca_crl_distribution_points_missing': 6668, 'e_sub_cert_valid_time_longer_than_39_months': 6611, 'e_dnsname_not_valid_tld': 6284, 'e_sub_cert_eku_missing': 6206, 'e_sub_cert_valid_time_longer_than_825_days': 5740, 'e_ca_key_usage_missing': 4535, 'e_tls_server_cert_valid_time_longer_than_398_days': 3721, 'e_dnsname_bad_character_in_label': 3472, 'e_basic_constraints_not_critical': 3438, 'e_ca_country_name_missing': 2908, 'e_sub_cert_locality_name

In [6]:
# franken 8M
errors_franken = ['e_qcstatem_etsi_present_qcs_critical', 'e_cert_policy_ov_requires_province_or_locality', 'e_sub_cert_province_must_appear', 'e_ext_san_not_critical_without_subject', 'e_ev_country_name_missing', 'e_ext_cert_policy_explicit_text_too_long', 'e_ext_san_no_entries', 'e_subject_dn_not_printable_characters', 'e_ev_organization_name_missing', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_cert_aia_marked_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_rsa_mod_less_than_2048_bits', 'e_ext_name_constraints_not_in_ca', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_sub_cert_given_name_surname_contains_correct_policy', 'e_ext_san_rfc822_name_present', 'e_ca_organization_name_missing', 'e_sub_ca_crl_distribution_points_missing', 'e_ext_ian_no_entries', 'e_ext_subject_key_identifier_critical', 'e_cab_iv_requires_personal_name', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_ext_key_usage_without_bits', 'e_path_len_constraint_improperly_included', 'e_invalid_certificate_version', 'e_qcstatem_mandatory_etsi_statems', 'e_validity_time_not_positive', 'e_ext_authority_key_identifier_critical', 'e_dnsname_empty_label', 'e_dnsname_not_valid_tld', 'e_cert_policy_ov_requires_country', 'e_cert_extensions_version_not_3', 'e_dnsname_contains_bare_iana_suffix', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_valid_time_longer_than_825_days', 'e_subject_country_not_iso', 'e_sub_ca_aia_missing', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_sub_cert_eku_missing', 'e_ca_key_cert_sign_not_set', 'e_wrong_time_format_pre2050', 'e_root_ca_key_usage_present', 'e_sub_cert_cert_policy_empty', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_ext_subject_directory_attr_critical', 'e_ext_san_empty_name', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ev_serial_number_missing', 'e_cert_policy_iv_requires_province_or_locality', 'e_sub_cert_locality_name_must_appear', 'e_cab_dv_conflicts_with_org', 'e_ca_crl_sign_not_set', 'e_san_bare_wildcard', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_san_uri_relative', 'e_sub_cert_certificate_policies_missing', 'e_ev_business_category_missing', 'e_issuer_field_empty', 'e_ext_san_uri_format_invalid', 'e_root_ca_key_usage_must_be_critical', 'e_cert_policy_iv_requires_country', 'e_cab_dv_conflicts_with_postal', 'e_cab_dv_conflicts_with_street', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_mp_authority_key_identifier_correct', 'e_sub_cert_not_is_ca', 'e_dnsname_label_too_long', 'e_subject_common_name_not_from_san', 'e_ext_san_missing', 'e_san_wildcard_not_first', 'e_root_ca_extended_key_usage_present', 'e_ext_name_constraints_not_critical', 'e_dnsname_left_label_wildcard_correct', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_locality', 'e_basic_constraints_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_ext_san_directory_name_present', 'e_ca_key_usage_not_critical', 'e_ca_key_usage_missing', 'e_dnsname_underscore_in_sld', 'e_subject_empty_without_san', 'e_subject_contains_noninformational_value', 'e_ca_is_ca', 'e_ext_authority_key_identifier_missing', 'e_ca_country_name_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_sub_cert_locality_name_must_not_appear', 'e_sub_ca_aia_marked_critical', 'e_subject_organizational_unit_name_max_length', 'e_ca_subject_field_empty', 'e_cab_ov_requires_org', 'e_ext_subject_key_identifier_missing_ca', 'e_dnsname_wildcard_only_in_left_label', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_issuer_dn_country_not_printable_string', 'e_ev_valid_time_too_long', 'e_subject_printable_string_badalpha', 'e_dnsname_bad_character_in_label', 'e_subject_contains_reserved_ip', 'e_ca_common_name_missing', 'e_ext_san_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_subject_common_name_max_length', 'e_ext_aia_marked_critical', 'e_cab_dv_conflicts_with_province', 'e_sub_cert_country_name_must_appear', 'e_sub_ca_certificate_policies_missing', 'e_subject_dn_country_not_printable_string', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_sub_cert_aia_missing', 'e_ca_country_name_invalid', 'e_ext_san_other_name_present', 'e_sub_cert_valid_time_longer_than_39_months']
# transcert 30K
errors_transcert = ['e_cert_policy_ov_requires_province_or_locality', 'e_sub_cert_province_must_appear', 'e_ext_duplicate_extension', 'e_ext_san_not_critical_without_subject', 'e_ev_country_name_missing', 'e_ev_organization_name_missing', 'e_sub_cert_aia_marked_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_ext_name_constraints_not_in_ca', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_ext_san_rfc822_name_present', 'e_ca_organization_name_missing', 'e_sub_ca_crl_distribution_points_missing', 'e_ext_subject_key_identifier_critical', 'e_cab_iv_requires_personal_name', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_ext_key_usage_without_bits', 'e_path_len_constraint_improperly_included', 'e_invalid_certificate_version', 'e_validity_time_not_positive', 'e_ext_authority_key_identifier_critical', 'e_dnsname_not_valid_tld', 'e_cert_policy_ov_requires_country', 'e_cert_extensions_version_not_3', 'e_dnsname_contains_bare_iana_suffix', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_valid_time_longer_than_825_days', 'e_subject_country_not_iso', 'e_sub_ca_aia_missing', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_sub_cert_eku_missing', 'e_ca_key_cert_sign_not_set', 'e_root_ca_key_usage_present', 'e_wrong_time_format_pre2050', 'e_sub_cert_cert_policy_empty', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ev_serial_number_missing', 'e_cert_policy_iv_requires_province_or_locality', 'e_sub_cert_locality_name_must_appear', 'e_cab_dv_conflicts_with_org', 'e_ca_crl_sign_not_set', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_sub_cert_certificate_policies_missing', 'e_ev_business_category_missing', 'e_issuer_field_empty', 'e_root_ca_key_usage_must_be_critical', 'e_cert_policy_iv_requires_country', 'e_cab_dv_conflicts_with_street', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_mp_authority_key_identifier_correct', 'e_sub_cert_not_is_ca', 'e_subject_common_name_not_from_san', 'e_ext_san_missing', 'e_ext_name_constraints_not_critical', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_locality', 'e_basic_constraints_not_critical', 'e_ext_san_directory_name_present', 'e_ca_key_usage_not_critical', 'e_ca_key_usage_missing', 'e_subject_empty_without_san', 'e_subject_contains_noninformational_value', 'e_ca_is_ca', 'e_ca_country_name_missing', 'e_ext_authority_key_identifier_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_sub_cert_locality_name_must_not_appear', 'e_sub_ca_aia_marked_critical', 'e_cab_ov_requires_org', 'e_ca_subject_field_empty', 'e_ext_subject_key_identifier_missing_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_issuer_dn_country_not_printable_string', 'e_ev_valid_time_too_long', 'e_dnsname_bad_character_in_label', 'e_ca_common_name_missing', 'e_subject_contains_reserved_ip', 'e_ext_san_contains_reserved_ip', 'e_ext_aia_marked_critical', 'e_cab_dv_conflicts_with_province', 'e_sub_ca_certificate_policies_missing', 'e_sub_cert_country_name_must_appear', 'e_subject_dn_country_not_printable_string', 'e_sub_cert_aia_missing', 'e_ca_country_name_invalid', 'e_ext_san_other_name_present', 'e_sub_cert_valid_time_longer_than_39_months']

discrepancies_to_zlints = {}

for f in noDiscrepancies:
    if fileToDisc[f] not in discrepancies_to_zlints:
        discrepancies_to_zlints[fileToDisc[f]] = set()

    for lint in file_to_error[f]:
        discrepancies_to_zlints[fileToDisc[f]].add(lint)

lints_seen = set()
print("Total discrepancies", len(discrepancies_to_zlints))
for disc in discrepancies_to_zlints:
    local_lints = set()
    for lint in discrepancies_to_zlints[disc]:
        # if lint in errors_franken:
        #     continue

        # if lint in errors_transcert:
        #     continue
        
        # if lint not in lints_seen:
        #     local_lints.add(lint)
        #     lints_seen.add(lint)    

        local_lints.add(lint)
        lints_seen.add(lint)    
    
    #print(disc, len(discrepancies_to_zlints[disc]), list(discrepancies_to_zlints[disc])[0], "...")
    print(disc, len(local_lints))

Total discrepancies 26
01010 45
00001 85
00011 78
11011 88
01101 54
01011 68
01111 94
11110 73
01000 65
00111 72
11001 54
01001 63
00100 31
11010 55
11101 45
00101 27
10010 19
10001 32
00010 54
10011 45
01110 45
11000 28
01100 38
10000 35
10111 14
11100 10


In [7]:
# print("Number of certs unable to run zlint on:", len(no_zlint))

# errors_zmap_original = ['e_ca_common_name_missing', 'e_invalid_certificate_version', 'e_ext_san_rfc822_name_present', 'e_ca_key_usage_missing', 'e_sub_ca_certificate_policies_missing', 'e_cert_extensions_version_not_3', 'e_sub_cert_province_must_appear', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_ext_san_contains_reserved_ip', 'e_sub_cert_certificate_policies_missing', 'e_subject_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_too_long', 'e_rsa_mod_less_than_2048_bits', 'e_sub_ca_crl_distribution_points_missing', 'e_dnsname_label_too_long', 'e_sub_cert_locality_name_must_appear', 'e_subject_contains_noninformational_value', 'e_sub_cert_or_sub_ca_using_sha1', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ca_is_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_qcstatem_mandatory_etsi_statems', 'e_subject_country_not_iso', 'e_root_ca_key_usage_present', 'e_subject_printable_string_badalpha', 'e_issuer_dn_country_not_printable_string', 'e_cab_ov_requires_org', 'e_wrong_time_format_pre2050', 'e_mp_authority_key_identifier_correct', 'e_mp_ecdsa_signature_encoding_correct', 'e_ev_business_category_missing', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_ext_san_uri_format_invalid', 'e_subject_organizational_unit_name_max_length', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_subject_common_name_max_length', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_ca_aia_missing', 'e_ext_san_directory_name_present', 'e_cert_sig_alg_not_match_tbs_sig_alg', 'e_ca_key_cert_sign_not_set', 'e_san_bare_wildcard', 'e_ext_authority_key_identifier_missing', 'e_ev_valid_time_too_long', 'e_ext_san_uri_relative', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_sub_cert_valid_time_longer_than_39_months', 'e_utc_time_does_not_include_seconds', 'e_dnsname_wildcard_only_in_left_label', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_ext_san_missing', 'e_ca_organization_name_missing', 'e_signature_algorithm_not_supported', 'e_subject_common_name_not_from_san', 'e_sub_cert_cert_policy_empty', 'e_dnsname_left_label_wildcard_correct', 'e_ext_san_empty_name', 'e_ext_subject_key_identifier_missing_ca', 'e_dnsname_bad_character_in_label', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_ian_no_entries', 'e_sub_cert_aia_missing', 'e_mp_ecdsa_pub_key_encoding_correct', 'e_ext_key_usage_cert_sign_without_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_sub_cert_country_name_must_appear', 'e_dnsname_empty_label', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_eku_missing', 'e_sub_cert_valid_time_longer_than_825_days', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_path_len_constraint_improperly_included', 'e_validity_time_not_positive', 'e_tbs_signature_rsa_encryption_parameter_not_null', 'e_root_ca_key_usage_must_be_critical', 'e_ext_name_constraints_not_critical', 'e_dnsname_underscore_in_sld', 'e_ca_country_name_invalid', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_ev_serial_number_missing', 'e_ext_san_no_entries', 'e_cert_policy_ov_requires_country', 'e_serial_number_longer_than_20_octets', 'e_dnsname_not_valid_tld', 'e_sub_cert_locality_name_must_not_appear', 'e_san_wildcard_not_first', 'e_dnsname_contains_bare_iana_suffix', 'e_mp_modulus_must_be_divisible_by_8', 'e_subject_dn_serial_number_not_printable_string', 'e_sub_cert_not_is_ca', 'e_basic_constraints_not_critical', 'e_ca_key_usage_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_subject_dn_not_printable_characters', 'e_ext_san_other_name_present', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_country_not_printable_string', 'e_cert_policy_ov_requires_province_or_locality', 'e_ca_crl_sign_not_set', 'e_serial_number_not_positive', 'e_ca_country_name_missing']
# #errors_franken = ['e_ca_common_name_missing', 'e_cab_dv_conflicts_with_street', 'e_invalid_certificate_version', 'e_ext_san_rfc822_name_present', 'e_cab_dv_conflicts_with_locality', 'e_ca_key_usage_missing', 'e_sub_ca_certificate_policies_missing', 'e_ext_authority_key_identifier_critical', 'e_cab_dv_conflicts_with_org', 'e_cert_extensions_version_not_3', 'e_sub_cert_province_must_appear', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_ext_san_contains_reserved_ip', 'e_sub_cert_certificate_policies_missing', 'e_subject_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_too_long', 'e_cab_iv_requires_personal_name', 'e_rsa_mod_less_than_2048_bits', 'e_sub_ca_crl_distribution_points_missing', 'e_dnsname_label_too_long', 'e_sub_cert_locality_name_must_appear', 'e_subject_contains_noninformational_value', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ca_is_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_cab_dv_conflicts_with_province', 'e_qcstatem_mandatory_etsi_statems', 'e_subject_country_not_iso', 'e_ext_subject_directory_attr_critical', 'e_root_ca_key_usage_present', 'e_ev_organization_name_missing', 'e_ext_aia_marked_critical', 'e_cab_ov_requires_org', 'e_issuer_dn_country_not_printable_string', 'e_subject_printable_string_badalpha', 'e_wrong_time_format_pre2050', 'e_mp_authority_key_identifier_correct', 'e_sub_ca_aia_marked_critical', 'e_ev_business_category_missing', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_ext_subject_key_identifier_critical', 'e_sub_cert_aia_marked_critical', 'e_subject_organizational_unit_name_max_length', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_ca_aia_missing', 'e_ext_san_directory_name_present', 'e_ca_key_cert_sign_not_set', 'e_san_bare_wildcard', 'e_ext_authority_key_identifier_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_ev_valid_time_too_long', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_sub_cert_valid_time_longer_than_39_months', 'e_dnsname_wildcard_only_in_left_label', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_cert_policy_iv_requires_country', 'e_ext_san_missing', 'e_ext_name_constraints_not_in_ca', 'e_ca_organization_name_missing', 'e_subject_common_name_not_from_san', 'e_sub_cert_cert_policy_empty', 'e_ext_san_empty_name', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_ext_subject_key_identifier_missing_ca', 'e_ext_key_usage_without_bits', 'e_dnsname_bad_character_in_label', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_ian_no_entries', 'e_sub_cert_aia_missing', 'e_qcstatem_etsi_present_qcs_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_sub_cert_country_name_must_appear', 'e_dnsname_empty_label', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_eku_missing', 'e_sub_cert_valid_time_longer_than_825_days', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_ev_country_name_missing', 'e_path_len_constraint_improperly_included', 'e_validity_time_not_positive', 'e_cert_policy_iv_requires_province_or_locality', 'e_ext_name_constraints_not_critical', 'e_dnsname_underscore_in_sld', 'e_root_ca_key_usage_must_be_critical', 'e_ca_country_name_invalid', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_ev_serial_number_missing', 'e_cert_policy_ov_requires_country', 'e_sub_cert_given_name_surname_contains_correct_policy', 'e_dnsname_not_valid_tld', 'e_sub_cert_locality_name_must_not_appear', 'e_dnsname_contains_bare_iana_suffix', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_postal', 'e_sub_cert_not_is_ca', 'e_basic_constraints_not_critical', 'e_ca_key_usage_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_subject_dn_not_printable_characters', 'e_ext_san_other_name_present', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_country_not_printable_string', 'e_cert_policy_ov_requires_province_or_locality', 'e_subject_empty_without_san', 'e_ca_crl_sign_not_set', 'e_ca_country_name_missing']
# # franken 1M
# errors_franken = ['e_ca_common_name_missing', 'e_cab_dv_conflicts_with_street', 'e_invalid_certificate_version', 'e_ext_san_rfc822_name_present', 'e_cab_dv_conflicts_with_locality', 'e_ca_key_usage_missing', 'e_sub_ca_certificate_policies_missing', 'e_ext_authority_key_identifier_critical', 'e_cab_dv_conflicts_with_org', 'e_cert_extensions_version_not_3', 'e_sub_cert_province_must_appear', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_ext_san_contains_reserved_ip', 'e_sub_cert_certificate_policies_missing', 'e_subject_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_too_long', 'e_cab_iv_requires_personal_name', 'e_rsa_mod_less_than_2048_bits', 'e_sub_ca_crl_distribution_points_missing', 'e_dnsname_label_too_long', 'e_sub_cert_locality_name_must_appear', 'e_subject_contains_noninformational_value', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ca_is_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_cab_dv_conflicts_with_province', 'e_qcstatem_mandatory_etsi_statems', 'e_subject_country_not_iso', 'e_ext_subject_directory_attr_critical', 'e_root_ca_key_usage_present', 'e_issuer_dn_country_not_printable_string', 'e_ext_aia_marked_critical', 'e_ev_organization_name_missing', 'e_subject_printable_string_badalpha', 'e_cab_ov_requires_org', 'e_wrong_time_format_pre2050', 'e_mp_authority_key_identifier_correct', 'e_sub_ca_aia_marked_critical', 'e_ev_business_category_missing', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_ext_subject_key_identifier_critical', 'e_sub_cert_aia_marked_critical', 'e_subject_organizational_unit_name_max_length', 'e_ext_san_uri_format_invalid', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_subject_common_name_max_length', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_ca_aia_missing', 'e_ext_san_directory_name_present', 'e_ca_key_cert_sign_not_set', 'e_san_bare_wildcard', 'e_ext_authority_key_identifier_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_ev_valid_time_too_long', 'e_ext_san_uri_relative', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_sub_cert_valid_time_longer_than_39_months', 'e_dnsname_wildcard_only_in_left_label', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_cert_policy_iv_requires_country', 'e_ext_san_missing', 'e_ext_name_constraints_not_in_ca', 'e_ca_organization_name_missing', 'e_subject_common_name_not_from_san', 'e_sub_cert_cert_policy_empty', 'e_dnsname_left_label_wildcard_correct', 'e_ext_san_empty_name', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_ext_subject_key_identifier_missing_ca', 'e_ext_key_usage_without_bits', 'e_dnsname_bad_character_in_label', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_ian_no_entries', 'e_sub_cert_aia_missing', 'e_qcstatem_etsi_present_qcs_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_sub_cert_country_name_must_appear', 'e_dnsname_empty_label', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_eku_missing', 'e_sub_cert_valid_time_longer_than_825_days', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_ev_country_name_missing', 'e_path_len_constraint_improperly_included', 'e_validity_time_not_positive', 'e_cert_policy_iv_requires_province_or_locality', 'e_ext_name_constraints_not_critical', 'e_dnsname_underscore_in_sld', 'e_root_ca_key_usage_must_be_critical', 'e_ca_country_name_invalid', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_ev_serial_number_missing', 'e_ext_san_no_entries', 'e_cert_policy_ov_requires_country', 'e_sub_cert_given_name_surname_contains_correct_policy', 'e_dnsname_not_valid_tld', 'e_sub_cert_locality_name_must_not_appear', 'e_san_wildcard_not_first', 'e_dnsname_contains_bare_iana_suffix', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_postal', 'e_sub_cert_not_is_ca', 'e_ca_subject_field_empty', 'e_basic_constraints_not_critical', 'e_ca_key_usage_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_subject_dn_not_printable_characters', 'e_ext_san_not_critical_without_subject', 'e_ext_san_other_name_present', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_country_not_printable_string', 'e_cert_policy_ov_requires_province_or_locality', 'e_subject_empty_without_san', 'e_issuer_field_empty', 'e_ca_crl_sign_not_set', 'e_ca_country_name_missing']
# # transcert 30K
# errors_transcert = []

# errors_no_disc = set(all_errors)

# total_lints = 267
# print(len(set(all_errors)))
# for x in collections.Counter(all_errors).most_common():
#     print(x)

# print("*** NOT FOUND IN FRANKEN OR ZMAP :", len(errors_no_disc.difference(errors_franken).difference(errors_zmap_original)))
# count = 0
# for x in errors_no_disc.difference(errors_franken).difference(errors_zmap_original):
#     count += 1
#     print(count , x, len(all_errors_source[x]))
# #print(all_errors_source['e_ext_cert_policy_disallowed_any_policy_qualifier'])

print("Number of certs unable to run zlint on:", len(no_zlint))

# errors_zmap_original = ['e_ca_common_name_missing', 'e_invalid_certificate_version', 'e_ext_san_rfc822_name_present', 'e_ca_key_usage_missing', 'e_sub_ca_certificate_policies_missing', 'e_cert_extensions_version_not_3', 'e_sub_cert_province_must_appear', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_ext_san_contains_reserved_ip', 'e_sub_cert_certificate_policies_missing', 'e_subject_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_too_long', 'e_rsa_mod_less_than_2048_bits', 'e_sub_ca_crl_distribution_points_missing', 'e_dnsname_label_too_long', 'e_sub_cert_locality_name_must_appear', 'e_subject_contains_noninformational_value', 'e_sub_cert_or_sub_ca_using_sha1', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ca_is_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_qcstatem_mandatory_etsi_statems', 'e_subject_country_not_iso', 'e_root_ca_key_usage_present', 'e_subject_printable_string_badalpha', 'e_issuer_dn_country_not_printable_string', 'e_cab_ov_requires_org', 'e_wrong_time_format_pre2050', 'e_mp_authority_key_identifier_correct', 'e_mp_ecdsa_signature_encoding_correct', 'e_ev_business_category_missing', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_ext_san_uri_format_invalid', 'e_subject_organizational_unit_name_max_length', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_subject_common_name_max_length', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_ca_aia_missing', 'e_ext_san_directory_name_present', 'e_cert_sig_alg_not_match_tbs_sig_alg', 'e_ca_key_cert_sign_not_set', 'e_san_bare_wildcard', 'e_ext_authority_key_identifier_missing', 'e_ev_valid_time_too_long', 'e_ext_san_uri_relative', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_sub_cert_valid_time_longer_than_39_months', 'e_utc_time_does_not_include_seconds', 'e_dnsname_wildcard_only_in_left_label', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_ext_san_missing', 'e_ca_organization_name_missing', 'e_signature_algorithm_not_supported', 'e_subject_common_name_not_from_san', 'e_sub_cert_cert_policy_empty', 'e_dnsname_left_label_wildcard_correct', 'e_ext_san_empty_name', 'e_ext_subject_key_identifier_missing_ca', 'e_dnsname_bad_character_in_label', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_ian_no_entries', 'e_sub_cert_aia_missing', 'e_mp_ecdsa_pub_key_encoding_correct', 'e_ext_key_usage_cert_sign_without_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_sub_cert_country_name_must_appear', 'e_dnsname_empty_label', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_eku_missing', 'e_sub_cert_valid_time_longer_than_825_days', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_path_len_constraint_improperly_included', 'e_validity_time_not_positive', 'e_tbs_signature_rsa_encryption_parameter_not_null', 'e_root_ca_key_usage_must_be_critical', 'e_ext_name_constraints_not_critical', 'e_dnsname_underscore_in_sld', 'e_ca_country_name_invalid', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_ev_serial_number_missing', 'e_ext_san_no_entries', 'e_cert_policy_ov_requires_country', 'e_serial_number_longer_than_20_octets', 'e_dnsname_not_valid_tld', 'e_sub_cert_locality_name_must_not_appear', 'e_san_wildcard_not_first', 'e_dnsname_contains_bare_iana_suffix', 'e_mp_modulus_must_be_divisible_by_8', 'e_subject_dn_serial_number_not_printable_string', 'e_sub_cert_not_is_ca', 'e_basic_constraints_not_critical', 'e_ca_key_usage_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_subject_dn_not_printable_characters', 'e_ext_san_other_name_present', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_country_not_printable_string', 'e_cert_policy_ov_requires_province_or_locality', 'e_ca_crl_sign_not_set', 'e_serial_number_not_positive', 'e_ca_country_name_missing']
#errors_franken = ['e_ca_common_name_missing', 'e_cab_dv_conflicts_with_street', 'e_invalid_certificate_version', 'e_ext_san_rfc822_name_present', 'e_cab_dv_conflicts_with_locality', 'e_ca_key_usage_missing', 'e_sub_ca_certificate_policies_missing', 'e_ext_authority_key_identifier_critical', 'e_cab_dv_conflicts_with_org', 'e_cert_extensions_version_not_3', 'e_sub_cert_province_must_appear', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_ext_san_contains_reserved_ip', 'e_sub_cert_certificate_policies_missing', 'e_subject_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_too_long', 'e_cab_iv_requires_personal_name', 'e_rsa_mod_less_than_2048_bits', 'e_sub_ca_crl_distribution_points_missing', 'e_dnsname_label_too_long', 'e_sub_cert_locality_name_must_appear', 'e_subject_contains_noninformational_value', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ca_is_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_cab_dv_conflicts_with_province', 'e_qcstatem_mandatory_etsi_statems', 'e_subject_country_not_iso', 'e_ext_subject_directory_attr_critical', 'e_root_ca_key_usage_present', 'e_ev_organization_name_missing', 'e_ext_aia_marked_critical', 'e_cab_ov_requires_org', 'e_issuer_dn_country_not_printable_string', 'e_subject_printable_string_badalpha', 'e_wrong_time_format_pre2050', 'e_mp_authority_key_identifier_correct', 'e_sub_ca_aia_marked_critical', 'e_ev_business_category_missing', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_ext_subject_key_identifier_critical', 'e_sub_cert_aia_marked_critical', 'e_subject_organizational_unit_name_max_length', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_ca_aia_missing', 'e_ext_san_directory_name_present', 'e_ca_key_cert_sign_not_set', 'e_san_bare_wildcard', 'e_ext_authority_key_identifier_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_ev_valid_time_too_long', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_sub_cert_valid_time_longer_than_39_months', 'e_dnsname_wildcard_only_in_left_label', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_cert_policy_iv_requires_country', 'e_ext_san_missing', 'e_ext_name_constraints_not_in_ca', 'e_ca_organization_name_missing', 'e_subject_common_name_not_from_san', 'e_sub_cert_cert_policy_empty', 'e_ext_san_empty_name', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_ext_subject_key_identifier_missing_ca', 'e_ext_key_usage_without_bits', 'e_dnsname_bad_character_in_label', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_ian_no_entries', 'e_sub_cert_aia_missing', 'e_qcstatem_etsi_present_qcs_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_sub_cert_country_name_must_appear', 'e_dnsname_empty_label', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_eku_missing', 'e_sub_cert_valid_time_longer_than_825_days', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_ev_country_name_missing', 'e_path_len_constraint_improperly_included', 'e_validity_time_not_positive', 'e_cert_policy_iv_requires_province_or_locality', 'e_ext_name_constraints_not_critical', 'e_dnsname_underscore_in_sld', 'e_root_ca_key_usage_must_be_critical', 'e_ca_country_name_invalid', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_ev_serial_number_missing', 'e_cert_policy_ov_requires_country', 'e_sub_cert_given_name_surname_contains_correct_policy', 'e_dnsname_not_valid_tld', 'e_sub_cert_locality_name_must_not_appear', 'e_dnsname_contains_bare_iana_suffix', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_postal', 'e_sub_cert_not_is_ca', 'e_basic_constraints_not_critical', 'e_ca_key_usage_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_subject_dn_not_printable_characters', 'e_ext_san_other_name_present', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_country_not_printable_string', 'e_cert_policy_ov_requires_province_or_locality', 'e_subject_empty_without_san', 'e_ca_crl_sign_not_set', 'e_ca_country_name_missing']
# franken 8M
errors_franken = ['e_qcstatem_etsi_present_qcs_critical', 'e_cert_policy_ov_requires_province_or_locality', 'e_sub_cert_province_must_appear', 'e_ext_san_not_critical_without_subject', 'e_ev_country_name_missing', 'e_ext_cert_policy_explicit_text_too_long', 'e_ext_san_no_entries', 'e_subject_dn_not_printable_characters', 'e_ev_organization_name_missing', 'e_ext_san_uniform_resource_identifier_present', 'e_sub_cert_aia_marked_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_rsa_mod_less_than_2048_bits', 'e_ext_name_constraints_not_in_ca', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_sub_cert_given_name_surname_contains_correct_policy', 'e_ext_san_rfc822_name_present', 'e_ca_organization_name_missing', 'e_sub_ca_crl_distribution_points_missing', 'e_ext_ian_no_entries', 'e_ext_subject_key_identifier_critical', 'e_cab_iv_requires_personal_name', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_ext_key_usage_without_bits', 'e_path_len_constraint_improperly_included', 'e_invalid_certificate_version', 'e_qcstatem_mandatory_etsi_statems', 'e_validity_time_not_positive', 'e_ext_authority_key_identifier_critical', 'e_dnsname_empty_label', 'e_dnsname_not_valid_tld', 'e_cert_policy_ov_requires_country', 'e_cert_extensions_version_not_3', 'e_dnsname_contains_bare_iana_suffix', 'e_old_root_ca_rsa_mod_less_than_2048_bits', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_valid_time_longer_than_825_days', 'e_subject_country_not_iso', 'e_sub_ca_aia_missing', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_sub_cert_eku_missing', 'e_ca_key_cert_sign_not_set', 'e_wrong_time_format_pre2050', 'e_root_ca_key_usage_present', 'e_sub_cert_cert_policy_empty', 'e_ext_policy_constraints_not_critical', 'e_ext_san_uri_host_not_fqdn_or_ip', 'e_ext_subject_directory_attr_critical', 'e_ext_san_empty_name', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ev_serial_number_missing', 'e_cert_policy_iv_requires_province_or_locality', 'e_sub_cert_locality_name_must_appear', 'e_cab_dv_conflicts_with_org', 'e_ca_crl_sign_not_set', 'e_san_bare_wildcard', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_ext_san_uri_relative', 'e_sub_cert_certificate_policies_missing', 'e_ev_business_category_missing', 'e_issuer_field_empty', 'e_ext_san_uri_format_invalid', 'e_root_ca_key_usage_must_be_critical', 'e_cert_policy_iv_requires_country', 'e_cab_dv_conflicts_with_postal', 'e_cab_dv_conflicts_with_street', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_mp_authority_key_identifier_correct', 'e_sub_cert_not_is_ca', 'e_dnsname_label_too_long', 'e_subject_common_name_not_from_san', 'e_ext_san_missing', 'e_san_wildcard_not_first', 'e_root_ca_extended_key_usage_present', 'e_ext_name_constraints_not_critical', 'e_dnsname_left_label_wildcard_correct', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_locality', 'e_basic_constraints_not_critical', 'e_ext_nc_intersects_reserved_ip', 'e_ext_san_directory_name_present', 'e_ca_key_usage_not_critical', 'e_ca_key_usage_missing', 'e_dnsname_underscore_in_sld', 'e_subject_empty_without_san', 'e_subject_contains_noninformational_value', 'e_ca_is_ca', 'e_ext_authority_key_identifier_missing', 'e_ca_country_name_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_sub_cert_locality_name_must_not_appear', 'e_sub_ca_aia_marked_critical', 'e_subject_organizational_unit_name_max_length', 'e_ca_subject_field_empty', 'e_cab_ov_requires_org', 'e_ext_subject_key_identifier_missing_ca', 'e_dnsname_wildcard_only_in_left_label', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_issuer_dn_country_not_printable_string', 'e_ev_valid_time_too_long', 'e_subject_printable_string_badalpha', 'e_dnsname_bad_character_in_label', 'e_subject_contains_reserved_ip', 'e_ca_common_name_missing', 'e_ext_san_contains_reserved_ip', 'e_ext_cert_policy_explicit_text_ia5_string', 'e_subject_common_name_max_length', 'e_ext_aia_marked_critical', 'e_cab_dv_conflicts_with_province', 'e_sub_cert_country_name_must_appear', 'e_sub_ca_certificate_policies_missing', 'e_subject_dn_country_not_printable_string', 'e_mp_modulus_must_be_2048_bits_or_more', 'e_sub_cert_aia_missing', 'e_ca_country_name_invalid', 'e_ext_san_other_name_present', 'e_sub_cert_valid_time_longer_than_39_months']
# transcert 30K
errors_transcert = ['e_cert_policy_ov_requires_province_or_locality', 'e_sub_cert_province_must_appear', 'e_ext_duplicate_extension', 'e_ext_san_not_critical_without_subject', 'e_ev_country_name_missing', 'e_ev_organization_name_missing', 'e_sub_cert_aia_marked_critical', 'e_ext_key_usage_cert_sign_without_ca', 'e_tls_server_cert_valid_time_longer_than_398_days', 'e_ext_name_constraints_not_in_ca', 'e_sub_cert_crl_distribution_points_marked_critical', 'e_ext_san_rfc822_name_present', 'e_ca_organization_name_missing', 'e_sub_ca_crl_distribution_points_missing', 'e_ext_subject_key_identifier_critical', 'e_cab_iv_requires_personal_name', 'e_sub_cert_crl_distribution_points_does_not_contain_url', 'e_ext_key_usage_without_bits', 'e_path_len_constraint_improperly_included', 'e_invalid_certificate_version', 'e_validity_time_not_positive', 'e_ext_authority_key_identifier_critical', 'e_dnsname_not_valid_tld', 'e_cert_policy_ov_requires_country', 'e_cert_extensions_version_not_3', 'e_dnsname_contains_bare_iana_suffix', 'e_sub_cert_province_must_not_appear', 'e_sub_cert_valid_time_longer_than_825_days', 'e_subject_country_not_iso', 'e_sub_ca_aia_missing', 'e_sub_cert_aia_does_not_contain_ocsp_url', 'e_sub_cert_eku_missing', 'e_ca_key_cert_sign_not_set', 'e_root_ca_key_usage_present', 'e_wrong_time_format_pre2050', 'e_sub_cert_cert_policy_empty', 'e_ext_authority_key_identifier_no_key_identifier', 'e_ev_serial_number_missing', 'e_cert_policy_iv_requires_province_or_locality', 'e_sub_cert_locality_name_must_appear', 'e_cab_dv_conflicts_with_org', 'e_ca_crl_sign_not_set', 'e_sub_ca_crl_distribution_points_does_not_contain_url', 'e_sub_cert_certificate_policies_missing', 'e_ev_business_category_missing', 'e_issuer_field_empty', 'e_root_ca_key_usage_must_be_critical', 'e_cert_policy_iv_requires_country', 'e_cab_dv_conflicts_with_street', 'e_sub_cert_key_usage_crl_sign_bit_set', 'e_mp_authority_key_identifier_correct', 'e_sub_cert_not_is_ca', 'e_subject_common_name_not_from_san', 'e_ext_san_missing', 'e_ext_name_constraints_not_critical', 'e_root_ca_extended_key_usage_present', 'e_subject_dn_serial_number_not_printable_string', 'e_cab_dv_conflicts_with_locality', 'e_basic_constraints_not_critical', 'e_ext_san_directory_name_present', 'e_ca_key_usage_not_critical', 'e_ca_key_usage_missing', 'e_subject_empty_without_san', 'e_subject_contains_noninformational_value', 'e_ca_is_ca', 'e_ca_country_name_missing', 'e_ext_authority_key_identifier_missing', 'e_sub_ca_crl_distribution_points_marked_critical', 'e_sub_cert_locality_name_must_not_appear', 'e_sub_ca_aia_marked_critical', 'e_cab_ov_requires_org', 'e_ca_subject_field_empty', 'e_ext_subject_key_identifier_missing_ca', 'e_sub_cert_key_usage_cert_sign_bit_set', 'e_issuer_dn_country_not_printable_string', 'e_ev_valid_time_too_long', 'e_dnsname_bad_character_in_label', 'e_ca_common_name_missing', 'e_subject_contains_reserved_ip', 'e_ext_san_contains_reserved_ip', 'e_ext_aia_marked_critical', 'e_cab_dv_conflicts_with_province', 'e_sub_ca_certificate_policies_missing', 'e_sub_cert_country_name_must_appear', 'e_subject_dn_country_not_printable_string', 'e_sub_cert_aia_missing', 'e_ca_country_name_invalid', 'e_ext_san_other_name_present', 'e_sub_cert_valid_time_longer_than_39_months']

errors_no_disc = set(all_errors)

total_lints = 267
print(len(set(all_errors)))
for x in collections.Counter(all_errors).most_common():
    print(x)

print("*** NOT FOUND IN FRANKEN OR ZMAP :", len(errors_no_disc.difference(errors_franken).difference(errors_transcert)))
count = 0
for x in errors_no_disc.difference(errors_franken).difference(errors_transcert):
    count += 1
    print(count, x)
    # print(count , x, len(all_errors_source[x]))
#print(all_errors_source['e_ext_cert_policy_disallowed_any_policy_qualifier'])

Number of certs unable to run zlint on: 0
116
('e_tbs_signature_rsa_encryption_parameter_not_null', 101815)
('e_spki_rsa_encryption_parameter_not_null', 101373)
('e_sub_cert_aia_does_not_contain_ocsp_url', 9197)
('e_ext_san_missing', 9039)
('e_sub_cert_aia_missing', 8837)
('e_sub_cert_cert_policy_empty', 8794)
('e_sub_cert_certificate_policies_missing', 8794)
('e_subject_common_name_not_from_san', 8499)
('e_ext_authority_key_identifier_no_key_identifier', 8492)
('e_ext_authority_key_identifier_missing', 8025)
('e_sub_ca_certificate_policies_missing', 6844)
('e_sub_ca_aia_missing', 6695)
('e_sub_ca_crl_distribution_points_missing', 6668)
('e_sub_cert_valid_time_longer_than_39_months', 6611)
('e_dnsname_not_valid_tld', 6284)
('e_sub_cert_eku_missing', 6206)
('e_sub_cert_valid_time_longer_than_825_days', 5740)
('e_ca_key_usage_missing', 4535)
('e_tls_server_cert_valid_time_longer_than_398_days', 3721)
('e_dnsname_bad_character_in_label', 3472)
('e_basic_constraints_not_critical', 3438)
('

In [24]:
# # Scan ZLint Error Descriptions
# import glob

# all_lints = glob.glob("/tmp/XYZ/zlint/v3/lints/*/*.go")
# all_lints_names_descriptions = {}

# for l in all_lints:
#     with open(l) as lF:
#         x = lF.read()
#         if "LintMetadata: lint.LintMetadata{" in x:
#             name =  x[x.index("Name: ")+6:x.index("Description: ")].replace('"', '').replace(',','').strip()
#             description = x[x.index("Description: ")+13:x.index("Citation:")].replace('"', '').replace(',','').strip()
#             assert name not in all_lints_names_descriptions
#             all_lints_names_descriptions[name] = description
# print(len(all_lints_names_descriptions))

In [25]:
# # DISCLOSURE EMAILS
# import re

# json_result_dir = '/tmp/XYZ/testing-results/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/json/'
# pem_dir = '/tmp/XYZ/cert_data_pem/v3-experiments-extra/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5'
# output_pem_dir = '/tmp/XYZ/disclosures/xxx'
# log_result_dir = '/tmp/XYZ/testing-results/zmap-data-1024-3-0.0002lr-0.1dropout-epoch3-step300000T1.5/logs/'

# lints_failed = errors_no_disc.difference(errors_franken).difference(errors_zmap_original)
# for lint_f in lints_failed:
#     print("\n***")
#     print("\nZLint Error Name:", lint_f)
    
#     print("\nZLint Error Description:", all_lints_names_descriptions[lint_f])
#     for l in all_lints:
#         if l.endswith("lint_" + lint_f + "go"):
#             print(l)
    
#     #print(all_errors_source[lint_f][0])
#     # LIBRARY OUTPUTS
#     print("\nLibrary Outputs:")
#     with open(json_result_dir + "/output." + all_errors_source[lint_f][0].split("output.")[1].split(".pem")[0] + ".pem.json") as f:
#         fileX = f.read().replace("0", "accept")
#         print(re.sub('\d', 'reject', fileX))

#     # LIBRARY LOGS
#     with open(log_result_dir + "/output." + all_errors_source[lint_f][0].split("output.")[1].split(".pem")[0] + ".pem-openssl.log") as f:
#         lines = []
#         for l in f.readlines():
#             if l.startswith("***CHECKPOINT***"):
#                 continue
#             lines.append(l)
#         print(lines)

        
#     print("Certificate reference: ", all_errors_source[lint_f][0].split("output.")[1].split(".pem")[0])
    
# # import shutil
# # for lint_f in lints_failed:
# #     print("Certificate reference: ", all_errors_source[lint_f][0].split("output.")[1].split(".pem")[0])
# #     shutil.copyfile(pem_dir + "/output." + all_errors_source[lint_f][0].split("output.")[1].split(".pem")[0] + ".pem", output_pem_dir + "/output." + all_errors_source[lint_f][0].split("output.")[1].split(".pem")[0] + ".pem")